In [ ]:
#importing necessary packages

from pathlib import Path
from tempfile import NamedTemporaryFile
import fileinput
import os
import rdkit
import pandas as pd
import numpy as np
import mol2vec
from mol2vec import features
from mol2vec import helpers
from rdkit import Chem
from rdkit.Chem import Draw
import gensim
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import seaborn as sns
import matplotlib.pyplot as plt
import gensim.downloader
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import itertools
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score, LeaveOneOut, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ConstantKernel as C, RationalQuadratic
import numpy as np
import statistics

In [ ]:
'''
Three functions that assist in converting lists to strings and vice versa.
These are required in order to store vectors in csv files. 
'''


def str2float(string):
    split = list(string.split(','))
    floats_split = []
    for i in range(len(split)):
        floats = float(split[i])
        floats_split.append(floats)
    return floats_split

def stringToList(vectors):
    bracket_removed_mol2vec = []
    for i in range(len(vectors)):
        new_strings = vectors[i].replace('[', '')
        newer_strings = new_strings.replace(']', '')
        bracket_removed_mol2vec.append(newer_strings)

# Convert all vectors
    xList = []
    for i in range(len(bracket_removed_mol2vec)):
        float_vec = str2float(bracket_removed_mol2vec[i])
        xList.append(float_vec)
    
    return xList
    
def listToString(vectors):
    string_indices = []
    for i in range(len(vectors)):
        knn_string = ', '.join(str(k) for k in vectors[i])
        string_indices.append(knn_string)

    bracket_string_indices = []
    for i in range(len(string_indices)):
        bracket_string = '[' + string_indices[i] + ']'
        bracket_string_indices.append(bracket_string)
    
    return bracket_string_indices

In [ ]:
datasetPath = os.path.join(os.getcwd(), 'fullDataset.csv')
dataset = pd.read_csv(datasetPath)
smiles = list(dataset['smiles'])
vectors = stringToList(list(dataset['mol2vec']))

In [ ]:
'''
This function uses K-Means clustering with k = 10 to cluster the feature vectors
in the entire dataset. These must be inputted into the function.
It then determines the cluster that contains the most
detected molecules in IRAS 16293B. The SMILES strings and feature vectors
of the molecules that were also assigned to this "most common" cluster are returned
and saved in a csv filed titled 'commonCluster.csv'
'''

def runClustering(smiles, vectors):
    
    clusterModel = KMeans(n_clusters=10)
    clusterResult = clusterModel.fit(vectors)
    clusterLabels = clusterResult.labels_
    
    detectPath = os.path.join(os.getcwd(), 'final_detections.csv')
    detectUpload = pd.read_csv(detectPath)
    detectSmiles = list(detectUpload['smiles'])

    clusterDetection = []
    
    
    for i in range(len(smiles)):
        if smiles[i] in detectSmiles:
            clusterDetection.append(clusterLabels[idx])
    
    print(clusterDetection)
    mostCommonCluster = statistics.mode(clusterDetection)
    
    clusterVectors = []
    clusterSmiles = []
    for i in range(len(clusterLabels)):
        if clusterLabels[i] == mostCommonCluster:
            clusterVectors.append(vectors[i])
            clusterSmiles.append(smiles[i])
            
    clusterDF = pd.DataFrame()
    clusterDF['smiles'] = clusterSmiles
    clusterDF['vectors'] = clusterVectors
    
    savePath = os.path.join(os.getcwd(), 'commonCluster.csv')
    clusterDF.to_csv(savePath)
    
    return clusterSmiles, clusterVectors
